# RAG with Langchain and Llama
<hr style="border:2px solid black">

## 1. Introduction

#### Limitations of LLMs

- know nothing outside training data, e.g., up-to-date information, classified/private data
- not specialized in specfic use cases
- tend to hallucinate confidently, possibly leading to misinformation
- produce black box output: do not clarify what has led to the generation of particular content

#### Fine Tuning

- enhances model performance for specific use case through Transfer Learning with additional data
- changes model parameters, enhancing speed and reducing cost for specific task
- powelful tool for:
  + incorporating non-dynamic or past data
  + specific industries with nuances in writing style, vocabulary, terminology
- cutoff issue persists in absence of up-to-date information

#### Retrieval Augmented Generation (RAG)

- increases model capabilities through:
  + **retrieving** external and up-to-date information
  + **augmenting** the original prompt given to the model
  + **generating** response using context plus information
- ground llm model parameters remain unchanged (no Transfer Learning)
- powerful tool for making use of dynamic up-to-date information
- white box output: provides transprency behind the model without hallucination

#### RAG Framework

<img src="../../images/api_test/rag.png" width="950"/>


### Technology Stack

#### [LangChain](https://python.langchain.com/docs/introduction/)

> framework for developing applications powered by LLMs

#### [FAISS (Facebook AI Similarity Search)](https://ai.meta.com/tools/faiss/)

>  library allowing storage of contextual embedding vectors in vector database and similarity search

#### [Groq](https://groq.com/about-us/)

> engine providing fast AI inference (conclusion from brand new data) in the cloud

<hr style="border:2px solid black">

## 2. Warm Up

#### load credentials

In [12]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import IFixitLoader

load_dotenv()


True

In [13]:
groq_key = os.getenv('GROQ_KEY')
USER_AGENT = os.getenv("USER_AGENT")
print(groq_key)
print(USER_AGENT)

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
RepairAssistant/1.0 (https://github.com/MaJo632)


#### define llm

In [14]:
import warnings
warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=groq_key
)

#### define promt template

**What is a Prompt?**
>- set of instructions or input for an LLM provided by a user to guide its response
>- helps it understand the context and generate relevant and coherent language-based output

In [15]:
from langchain.prompts.prompt import PromptTemplate

In [16]:
query = """
    given the information {information} about a device to fix:
    1. Name the device, make, model and problem
    2. two steps of repairing
    """

In [17]:
prompt_template = PromptTemplate(
    input_variables=["information"],
    template=query
)

#### define Chain

**What is a Chain?**

> - allows to link the output of one LLM call as the input of another

In [18]:
chain = prompt_template | llm

**Note:**
The `|` symbol chains together the different components, feeding the output from one component as input into the next component.
In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model. 

#### invoke Chain

In [19]:
loader = IFixitLoader(
    "https://www.ifixit.com/Answers/View/318583/My+iPhone+6+is+typing+and+opening+apps+by+itself"
)
data = loader.load()

In [20]:
output = chain.invoke(input={"information": data})

In [21]:
print(output.content)

Based on the provided information, here are the answers to your questions:

**1. Name the device, make, model, and problem:**

* Device: iPhone
* Make: Apple
* Model: iPhone 6
* Problem: The iPhone 6 is typing and opening apps by itself, also known as "ghost touches" or "self-opening apps".

**2. Two steps of repairing:**

From the various answers provided, I've extracted two steps that seem to be effective in fixing the problem:

**Step 1: Clean the phone and check for dust and dirt**

* Hold the phone in portrait mode and twist it gently to dislodge any dust or dirt that may be affecting the screen's sensors.
* Rest the phone for 10 minutes to allow the dust to settle.
* Use compressed air or a soft cloth to clean the phone's ports and crevices.

**Step 2: Reset the phone and replace the screen or sensors**

* Reset the phone by holding the power and home buttons until the screen goes off and the Apple logo appears.
* If the problem persists, consider replacing the screen or sensors,

<hr style="border:2px solid black">